使用treelite包进行预测加速，可以让速度提升2-6倍
原理1：提前预知哪一个叶子节点被执行的可能性更大，进而可以提前执行子节点逻辑。
2：原始的分支比较可能会有浮点数比较逻辑，可以量化为数值比较逻辑
3.可以转化为so格式文件方便其他语言调用，并且避免参数泄漏

In [ ]:
import treelite
import numpy as np
import time
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=100000, n_features=1000)

In [ ]:
import xgboost
dtrain = xgboost.DMatrix(X, label=y)
params = {'max_depth':3, 'eta':1, 'objective':'reg:squarederror', 'eval_metric':'rmse'}
bst = xgboost.train(params, dtrain, 20, [(dtrain, 'train')])

In [ ]:
model = treelite.Model.from_xgboost(bst)

toolchain = 'gcc'   # change this value as necessary
model.export_lib(toolchain=toolchain, libpath='./mymodel.so', verbose=True)

In [ ]:
import treelite_runtime     # runtime module
predictor = treelite_runtime.Predictor('./mymodel.so', verbose=True)

In [ ]:
nrows = [1000, 10000, 100000, 200000]
xgb_time = []
tree_lite = []
for nrow in nrows:
    data = np.random.random((nrow, 1000))
    dtrain = xgboost.DMatrix(data)
    
    start = time.time()
    _ = bst.predict(dtrain)
    xgb_time.append(time.time() - start)
    
    batch = treelite_runtime.Batch.from_npy2d(data)
    
    start = time.time()
    _ = predictor.predict(batch)
    tree_lite.append(time.time() - start)

In [ ]:
import lightgbm as lgb

params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'max_depth':3,
    'metric': 'binary_logloss',
    'bagging_freq': 5,
    'verbose': 0
}

lgb_train = lgb.Dataset(X, y)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10)

gbm.save_model('model.txt')

In [ ]:
model = treelite.Model.load('model.txt', model_format='lightgbm')

toolchain = 'gcc'   # change this value as necessary
model.export_lib(toolchain=toolchain, libpath='./mymodel.so', verbose=True)

In [ ]:
predictor = treelite_runtime.Predictor('mymodel.so', verbose=True)

In [ ]:
data = treelite_runtime.DMatrix(x_train)
predictor.predict(data)